In [1]:
import pandas as pd

# Reading the dataset again after the reset
file_path = 'tidy_youtube_data.csv'
youtube_data = pd.read_csv(file_path, encoding="ISO-8859-1", index_col="rank")
youtube_data = youtube_data.rename(columns = {"country" : "Country"})

# Displaying the first few rows of the dataset
youtube_data.head()

,youtuber,subscribers,video_views,category,title,uploads,Country,abbreviation,channel_type,video_views_rank,...,subscribers_for_last_30_days,gross_tertiary_education_enrollment,population,unemployment_rate,urban_population,latitude,longitude,created_date,unemployment_rate_ordinal,education_enrollment_ordinal
rank,,,,,,,,,,,,,,,,,,,,,
1,T-Series,245000000,2.280000e+11,Music,T-Series,20082,India,IN,Music,1.0,...,2000000.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880,2006-03-13,Moderate,Low
2,YouTube Movies,170000000,0.000000e+00,Film & Animation,youtubemovies,1,United States,US,Games,4055159.0,...,NaN,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891,2006-03-05,High,High
3,MrBeast,166000000,2.836884e+10,Entertainment,MrBeast,741,United States,US,Entertainment,48.0,...,8000000.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891,2012-02-20,High,High
4,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,Cocomelon - Nursery Rhymes,966,United States,US,Education,2.0,...,1000000.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891,2006-09-01,High,High
5,SET India,159000000,1.480000e+11,Shows,SET India,116536,India,IN,Entertainment,3.0,...,1000000.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880,2006-09-20,Moderate,Low


In [2]:
country_ids = pd.read_csv('https://raw.githubusercontent.com/joelostblom/teaching-datasets/main/country-ids-and-continents.csv')
relevant_country_names = country_ids["Country"]
country_ids.head()

,ID,Country,Continent
0,4,Afghanistan,Asia
1,8,Albania,Europe
2,12,Algeria,Africa
3,24,Angola,Africa
4,10,Antarctica,Antarctica


In [3]:
youtube_data = youtube_data.merge(
    country_ids
)

In [4]:
# Filter the dataset to get the top 15 YouTube channels ranked by subscribers
top_channels = youtube_data.dropna().nlargest(15, 'subscribers')
top_channels.reset_index(drop=True, inplace = True)
top_channels.index = top_channels.index + 1  
# Display relevant columns for the top channels
top_channels[['youtuber', 'Country', 'population', 'urban_population', 'gross_tertiary_education_enrollment', 'subscribers', 'video_views']]


,youtuber,Country,population,urban_population,gross_tertiary_education_enrollment,subscribers,video_views
1,T-Series,India,1.366418e+09,471031528.0,28.1,245000000,2.280000e+11
2,MrBeast,United States,3.282395e+08,270663028.0,88.2,166000000,2.836884e+10
3,Cocomelon - Nursery Rhymes,United States,3.282395e+08,270663028.0,88.2,162000000,1.640000e+11
4,SET India,India,1.366418e+09,471031528.0,28.1,159000000,1.480000e+11
5,Like Nastya,Russia,1.443735e+08,107683889.0,81.9,106000000,9.047906e+10
6,Vlad and Niki,United States,3.282395e+08,270663028.0,88.2,98900000,7.718017e+10
7,Zee Music Company,India,1.366418e+09,471031528.0,28.1,96700000,5.785629e+10
8,WWE,United States,3.282395e+08,270663028.0,88.2,96000000,7.742847e+10
9,BLACKPINK,South Korea,5.170910e+07,42106719.0,94.3,89800000,3.214460e+10
10,Sony SAB,India,1.366418e+09,471031528.0,28.1,83000000,1.010000e+11


In [5]:
import altair as alt
# For the dropdown menu, we'll need a list of countries in the top_channels DataFrame
countries = top_channels['Country'].unique().tolist()

# Create the selection for the dropdown menu
select_country = alt.selection_single(
    name='Select', fields=['Country'],
    bind=alt.binding_select(options=countries)
)

# Plot A: Subscribers Visualization
plot_a = alt.Chart(top_channels).mark_bar().encode(
    y=alt.Y('youtuber:N', title='YouTube Channel', sort='-x'),
    x=alt.X('subscribers:Q', title='Number of Subscribers'),
    color=alt.condition(select_country, 'Country:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'subscribers', 'Country']
).properties(
    width=600,
    height=300,
    title='Top 15 YouTube Channels by Subscribers'
).add_selection(select_country)

# Plot B: Video Views Visualization
plot_b = alt.Chart(top_channels).mark_bar().encode(
    y=alt.Y('youtuber:N', title='YouTube Channel', sort='-x'),
    x=alt.X('video_views:Q', title='Video Views'),
    color=alt.condition(select_country, 'Country:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'video_views', 'Country']
).properties(
    width=600,
    height=300,
    title='Top 15 YouTube Channels by Video Views'
).add_selection(select_country)

combined_plots = alt.vconcat(plot_a, plot_b)

# Display the combined plots
combined_plots

C:\Users\dorager91\.conda\envs\dsci320\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
C:\Users\dorager91\.conda\envs\dsci320\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)

In [6]:
from vega_datasets import data
# Load the world map data
world = alt.topo_feature(data.world_110m.url, 'countries')

# Aggregate country statistics
country_stats = top_channels.groupby(['Country', 'ID']).agg({
    'population': 'mean',
    'urban_population': 'mean',
    'gross_tertiary_education_enrollment': 'mean',
    'subscribers': 'sum',
    'video_views': 'sum'
}).reset_index()


# Create the map visualization
map_chart = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color( 'Country:N'),
    tooltip=[
        alt.Tooltip('Country:N', title='Country'),
        alt.Tooltip('population:Q', title='Population'),
        alt.Tooltip('urban_population:Q', title='Urban Population'),
        alt.Tooltip('gross_tertiary_education_enrollment:Q', title='Gross Tertiary Education Enrollment'),
        alt.Tooltip('subscribers:Q', title='Subscribers'),
        alt.Tooltip('video_views:Q', title='Video Views')
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=country_stats, key='ID', fields=[
        'Country',
        'population',
        'urban_population',
        'gross_tertiary_education_enrollment',
        'subscribers',
        'video_views'
    ])
).project(
    type='equalEarth'
).properties(
    width=800,
    height=400,
    title='YouTube Channel Statistics by Country'
)

map_chart

alt.Chart(...)

In [7]:
# Create the selection for the dropdown menu
select_country = alt.selection_single(
    name='Select', fields=['Country'],
    bind=alt.binding_select(options=countries)
)

# Modify Plot A to include the dropdown menu selection
plot_a = alt.Chart(top_channels).mark_bar().encode(
    y=alt.Y('youtuber:N', title='YouTube Channel', sort='-x'),
    x=alt.X('subscribers:Q', title='Number of Subscribers'),
    color=alt.condition(select_country, 'Country:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'subscribers', 'Country']
).properties(
    width=200,
    height=200,
    title='Top 15 YouTube Channels by Subscribers'
).add_selection(select_country)

# Modify Plot B to include the dropdown menu selection
plot_b = alt.Chart(top_channels).mark_bar().encode(
    y=alt.Y('youtuber:N', title='YouTube Channel', sort='-x'),
    x=alt.X('video_views:Q', title='Video Views'),
    color=alt.condition(select_country, 'Country:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'video_views', 'Country']
).properties(
    width=200,
    height=200,
    title='Top 15 YouTube Channels by Video Views'
).add_selection(select_country)

# Modify the map chart to include the dropdown menu selection
map_chart = alt.Chart(world).mark_geoshape().encode(
    color=alt.condition(select_country, 'Country:N', alt.value('lightgray'), legend=None),
    tooltip=[
        alt.Tooltip('Country:N', title='Country'),
        alt.Tooltip('population:Q', title='Population'),
        alt.Tooltip('urban_population:Q', title='Urban Population'),
        alt.Tooltip('gross_tertiary_education_enrollment:Q', title='Gross Tertiary Education Enrollment'),
        alt.Tooltip('subscribers:Q', title='Subscribers'),
        alt.Tooltip('video_views:Q', title='Video Views')
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=country_stats, key='ID', fields=[
        'Country',
        'population',
        'urban_population',
        'gross_tertiary_education_enrollment',
        'subscribers',
        'video_views'
    ])
).project(
    type='equalEarth'
).properties(
    width=600,
    height=400,
    title='YouTube Channel Statistics by Country'
).add_selection(select_country)

# Combine the plots and map into a single dashboard
bar_charts = alt.vconcat(
    plot_a,
    plot_b
)

dashboard = alt.hconcat(
    bar_charts,
    map_chart
)

# Display the dashboard
dashboard


C:\Users\dorager91\.conda\envs\dsci320\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
C:\Users\dorager91\.conda\envs\dsci320\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)

In [8]:
top_channels[['youtuber', 'Country', 'channel_type', 'subscribers', 'video_views']]

,youtuber,Country,channel_type,subscribers,video_views
1,T-Series,India,Music,245000000,2.280000e+11
2,MrBeast,United States,Entertainment,166000000,2.836884e+10
3,Cocomelon - Nursery Rhymes,United States,Education,162000000,1.640000e+11
4,SET India,India,Entertainment,159000000,1.480000e+11
5,Like Nastya,Russia,People,106000000,9.047906e+10
6,Vlad and Niki,United States,Entertainment,98900000,7.718017e+10
7,Zee Music Company,India,Music,96700000,5.785629e+10
8,WWE,United States,Sports,96000000,7.742847e+10
9,BLACKPINK,South Korea,Music,89800000,3.214460e+10
10,Sony SAB,India,Entertainment,83000000,1.010000e+11


In [9]:
select_channel_type = alt.selection_single(
    name='Select', fields=['channel_type'],
    bind=alt.binding_select(options=top_channels['channel_type'].unique().tolist())
)

# Plot for Subscribers (positive X-axis)
plot_subscribers = alt.Chart(top_channels).mark_bar().encode(
    y=alt.Y('youtuber:N', title='YouTube Channel'),
    x=alt.X('subscribers:Q', title='Number of Subscribers'),
    color=alt.condition(select_channel_type, 'channel_type:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'subscribers', 'channel_type:N']
).add_selection(
    select_country
)

# Plot for Video Views (negative X-axis)
plot_views = alt.Chart(top_channels).mark_bar().encode(
    y=alt.Y('youtuber:N', axis=None),
    x=alt.X('video_views:Q', title='Video Views', scale=alt.Scale(reverse=True, zero=False)),
    color=alt.condition(select_channel_type, 'channel_type:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'subscribers', 'channel_type:N']
).add_selection(
    select_country
)

# Combining the plots with independent x-axis scales
combined_plots = alt.layer(plot_subscribers, plot_views).resolve_scale(
    y='shared',
    x='independent'  # Each plot keeps its own x-axis scale
).properties(
    width=600,
    height=300
)

# Display the combined plots
combined_plots


C:\Users\dorager91\.conda\envs\dsci320\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
C:\Users\dorager91\.conda\envs\dsci320\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

In [10]:
select_channel_type = alt.selection_single(
    name='Select', fields=['channel_type'],
    bind=alt.binding_select(options=top_channels['channel_type'].unique().tolist())
)

base = alt.Chart(top_channels).encode(
    y=alt.Y('youtuber:N', axis=None, sort=alt.EncodingSortField(field="subscribers", order='descending')),
    tooltip=['youtuber', 'channel_type'] 
).add_selection(
    select_channel_type 
)

# Plot for Subscribers (positive X-axis)
plot_subscribers = base.mark_bar().encode(
    x=alt.X('subscribers:Q', title='Number of Subscribers'),
    color=alt.condition(select_channel_type, 'channel_type:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'subscribers', 'channel_type'] 
)

# Plot for Video Views (negative X-axis)
plot_views = base.mark_bar().encode(
    x=alt.X('video_views:Q', title='Video Views', scale=alt.Scale(reverse=True)),
    color=alt.condition(select_channel_type, 'channel_type:N', alt.value('lightgray'), legend=None),
    tooltip=['youtuber', 'video_views', 'channel_type']
)

# Text overlay
text = base.mark_text(align='center', baseline='middle').encode(
    text='youtuber:N'
)

# Combining the plots
combined_plot1 = alt.hconcat(
    plot_views, text, plot_subscribers
).resolve_scale(
    y='shared',
    x='independent'
).add_selection(
    select_channel_type
)

combined_plot1


alt.HConcatChart(...)